In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from scipy.stats import norm
import time as ttt
import iisignature as iisig
from tqdm import *
from einops import rearrange

In [2]:
import torch
import torch.nn as nn
import pandas as pd 

In [3]:
x0 = 1.0 # initial condition
sigma = 1 # volatility
mu = 0.02
segs=20
r = 0.01 # risk free rate
batch_size = 1000 # batch size
steps=5000
T = 1 # maturity
dt = T/steps # mesh size
true = 0.5828174603130847# true option price

dt = T/steps # mesh size
dt_new = T/segs # new mesh after shrinkage
level = 3 # truncation level

In [4]:
batch_in=100
MOMENTUM = 0.99
EPSILON = 1e-6
import warnings
warnings.filterwarnings("ignore")

In [5]:
def create_stock2(x0,r,sigma,T,steps,n_path,dW):
    dt=T/steps; 
    sqrt_dt=np.sqrt(dt); 
    s_vec=[]; w_vec=[]; 
    w_vec.append(np.ones(n_path)*1e-6)
    s_vec.append(np.ones(n_path)*1.0)
    for i in range(steps): 
        w_vec.append(dW[:,i]) 
        s_vectemp=s_vec[-1]+ r*s_vec[-1]*dt+ w_vec[-1]*s_vec[-1]*sigma
        s_vec.append(s_vectemp)
    wvec=np.transpose(np.array(w_vec))
    BM_path=np.cumsum(wvec,axis=1)
    S_path=np.transpose(np.array(s_vec))
    return BM_path, S_path

def jointime(T,path): 
    n_path, steps=path.shape
    dt=T/(steps-1); 
    
    times=np.arange(0,T,dt)
    times=np.append(times,T); 
    times_vec=np.tile(times,[2,1]); 
    times_vec=np.transpose(times_vec)
    times_vec=np.tile(times_vec,[n_path,1,1])
    times_vec[:,:,1]=path
    return times_vec

def ComputeMultiLevelSig(path, number_of_segment, depth):
    n_batch, nsteps,n_path = path.shape
    t_vec = np.arange(0, nsteps-1, int(nsteps / number_of_segment))
    t_vec = np.append(t_vec, nsteps-1)
    MultiLevelSig = []
    
   # path_class=signatory.Path(path,depth);
    ll=iisig.sig(np.expand_dims(path[:,0,:],axis=1),depth)
    MultiLevelSig.append(ll)
    
    for i in range(len(t_vec)-1):    
        ## Notice that we only use the signature of the concatenation of time and space.
        MultiLevelSig.append(iisig.sig(path[:,0:t_vec[i+1]+1,:],depth)) ##if not
        #MultiLevelSig.append(path_class.signature(t_vec[i],t_vec[i+1]+1))
    MultiLevelSig=np.stack(MultiLevelSig)  
    MultiLevelSig=rearrange(MultiLevelSig, 'b c h -> c b h') 
    return MultiLevelSig

In [6]:
def generate_samples(batch_in=100):
    dW = np.sqrt(dt)*np.random.normal(size=(batch_in, steps))
    pth2=create_stock2(x0,r,sigma,T,steps,batch_in,dW)
    BM_timePath=jointime(T,pth2[0]); 
    S_timePath=jointime(T,pth2[1]);
    sigs=ComputeMultiLevelSig(S_timePath, 20, 3)
    selection = np.linspace(0,steps, segs+1, dtype = np.int)

    BM_seg=BM_timePath[:,selection,1]
    dW=BM_seg[:,1:]-BM_seg[:,:-1]
    dW=np.expand_dims(dW,axis=2)

    dW=torch.tensor(dW,dtype=torch.float32)
    sigs=torch.tensor(sigs,dtype=torch.float32)

    ss=S_timePath[:,:,1]
    YT=ss[:,-1]-np.min(ss,axis=1)
    YT=torch.tensor(YT,dtype=torch.float32)
    YT=YT.unsqueeze(axis=1)
    return sigs, dW, YT

In [7]:
class Config(object):
    n_layer = 4
    batch_size = 1024
    valid_size = 1024
    
    dim=14; 
    Ntime=20; 
    delta=1/Ntime
    sqrt_deltaT=np.sqrt(1.0/Ntime); 
    lam=1; 

    logging_frequency = 100
    verbose = True
    y_init_range = [0, 1]
    
    num_hiddens = [dim,64,64,1] ## 256 ,256
    
def get_config(name):
    try:
        return globals()[name]
    except KeyError:
        raise KeyError("config not defined.")

cfg=get_config('Config')

In [8]:
class Dense(nn.Module): 
    def __init__(self,cin, cout, batch_norm=False, activate=True): 
        super(Dense,self).__init__()
        self.cin=cin; 
        self.cout=cout; 
        self.activate=activate; 
        
        self.linear=nn.Linear(self.cin,self.cout) #The linear layer
        #BatchNorm1d: it requires the input to be a correct size
        if batch_norm: 
            self.bn=nn.BatchNorm1d(cout,eps=EPSILON,momentum=MOMENTUM)
        else: 
            self.bn=None
      #  nn.init.normal_(self.linear.weight,std=5.0/np.sqrt(cin+cout))
        # This is the He initialization
        
    def forward(self,x): 
        x=self.linear(x)
        if self.bn is not None:
            x=self.bn(x)
        if self.activate:
            x=torch.tanh(x)
        return x 
    
class FFN(nn.Module):
    def __init__(self, config):
        super(FFN,self).__init__()
        self.config=config
        
        self.bn=nn.BatchNorm1d(config.num_hiddens[0],eps=EPSILON,momentum=MOMENTUM) ## So there is batch norm no problem
        # range(1,5): 1,2,3,4
        self.layers=[Dense(config.num_hiddens[i-1],config.num_hiddens[i]) for i in range(1, len(config.num_hiddens)-1)]
        self.layers+=[Dense(config.num_hiddens[-2], config.num_hiddens[-1],activate=False)]
        self.layers=nn.Sequential(*self.layers)
    
    def forward(self,x):
      #  x=self.bn(x)
        x=self.layers(x)
        return x 
    
class Lookback_PPDE_Backward(nn.Module):
    def __init__(self,cfg): 
        super(Lookback_PPDE_Backward,self).__init__()
        self.cfg=cfg
        self.Ntime=self.cfg.Ntime 
     #   self.Y0=Parameter(torch.rand(100,1))
        #self.model=RNN2(input_size=14, output_size=1, hidden_size=128, num_layers=1)
        self.mList=nn.ModuleList([FFN(self.cfg) for _ in range(self.Ntime)])
        
    def forward(self,batch_sig,batch_dW,batch_YT): 
   #     Z_path=self.model(batch_sig)
        Y=batch_YT
        for i in np.arange(segs-1,-1, -1):
            Y=Y-Y*r*T/segs-sigma*self.mList[i](batch_sig[:,i,:])*batch_dW[:,i,:]
        return Y

In [9]:
def loss_var(x):
    temp=torch.var(x)
    return temp

In [10]:
import torch.optim as optim
from torch.nn import Parameter
import math
model_PPDE_bw=Lookback_PPDE_Backward(cfg)
model_PPDE_bw#.to(device)
optimizer=optim.Adam(model_PPDE_bw.parameters(),lr=1e-3)
grad_clip=0.1
#scheduler=torch.optim.lr_scheduler.StepLR(optimizer, step_size=80, gamma=0.1)

In [11]:
y0_mean=[];
loss_vec=[];

## 0.5823 -- 0.5784 -- 0.5862

In [12]:
for i in range(140):
    batch_x, batch_dw, batch_y =generate_samples(batch_in=1000)

    x_temp=model_PPDE_bw(batch_x,batch_dw,batch_y)
    loss_temp=loss_var(x_temp)
    if grad_clip: 
        nn.utils.clip_grad_value_(model_PPDE_bw.parameters(), grad_clip)

    optimizer.zero_grad()
    loss_temp.backward()
    optimizer.step()
#   scheduler.step()
    y0_val=x_temp.mean().cpu().detach().numpy()
    loss_val=loss_temp.cpu().detach().numpy()
    
    y0_mean.append(y0_val)
    loss_vec.append(loss_val)
    
    print("Iter:", i, 'The mean Y0 is', y0_val , 'Variance is:' ,loss_val)

Iter: 0 The mean Y0 is 0.527325 Variance is: 0.9707741
Iter: 1 The mean Y0 is 0.54906803 Variance is: 0.98697454
Iter: 2 The mean Y0 is 0.66612726 Variance is: 2.8840706
Iter: 3 The mean Y0 is 0.55211306 Variance is: 0.6277847
Iter: 4 The mean Y0 is 0.52717966 Variance is: 0.6581896
Iter: 5 The mean Y0 is 0.56897074 Variance is: 0.6982603
Iter: 6 The mean Y0 is 0.5390868 Variance is: 0.6883007
Iter: 7 The mean Y0 is 0.5343724 Variance is: 0.43374258
Iter: 8 The mean Y0 is 0.54068434 Variance is: 0.5365417
Iter: 9 The mean Y0 is 0.572184 Variance is: 0.7383763
Iter: 10 The mean Y0 is 0.63440394 Variance is: 1.3317533
Iter: 11 The mean Y0 is 0.591115 Variance is: 0.8487527
Iter: 12 The mean Y0 is 0.57090783 Variance is: 0.4332906
Iter: 13 The mean Y0 is 0.5604803 Variance is: 0.9869259
Iter: 14 The mean Y0 is 0.5787457 Variance is: 0.45177397
Iter: 15 The mean Y0 is 0.56740904 Variance is: 0.36447355
Iter: 16 The mean Y0 is 0.5753286 Variance is: 0.33024746
Iter: 17 The mean Y0 is 0.5956

In [17]:
y_pred=np.array(y0_mean)
loss_var=np.array(loss_vec)
iters=np.arange(0,len(y_pred),1)+1

In [18]:
df=pd.DataFrame()
df['iter']=iters
df['y_pred']=y_pred
df['loss_var']=loss_var

In [23]:
df.to_csv('eg1_trained_data/method2_2.csv')

In [24]:
start_idx=50
df[start_idx:][df[start_idx:].loss_var<1.0].y_pred.mean()

0.5796725153923035

In [50]:
np.mean(np.array([0.58-0,0.5784, 0.579, 0.578]))

0.57885

In [51]:
np.std(np.array([0.5823,0.5784,0.5862, 0.579,0.578]))

0.0031063805304566592

In [52]:
5.78-2*0.003,5.78+2*0.003

(5.774, 5.7860000000000005)